First, we navigate all directories within 'Parking_Lot_Vehicle_Images_UFPR04' to find and copy all images into a newly created directory named 'all_images'.

In [ ]:
import os
import numpy as np
import shutil

curr = os.getcwd()
newDir = os.path.join(curr,'all_images')
#print(newDir)

if not os.path.exists(newDir):
    os.makedirs(newDir)
    
imgPath = os.path.join(curr,'Parking_Lot_Vehicle_Images_UFPR04')
#Need to set dirUFPR04 to filepath of UFPR04 folder
dirUFPR04 = os.path.join(imgPath,'UFPR04')
#print(dirUFPR04)   


for mainDir in os.listdir(dirUFPR04):
    weatherDir = os.path.join(dirUFPR04, mainDir)
    for dateDir in os.listdir(weatherDir):
        occDir = os.path.join(weatherDir, dateDir)
        for dir in os.listdir(occDir):
            imageDir = os.path.join(occDir, dir)
            for file in os.listdir(imageDir):
                if file.endswith('.jpg'):
                    fullpath = os.path.join(imageDir, file)
                    shutil.copy(fullpath, newDir)


Now, we initialize a custom pytorch Dataset class named 'VehicleDataset' for our images.

In [ ]:
import torch
from skimage import io, transform
from PIL import Image
import natsort
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class VehicleDataset(Dataset):
    def __init__(self, main_dir, transform=None):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        #image = io.imread(img_loc)
        tensor_image = self.transform(image)
        return tensor_image

We then initialize the torchvision transforms object that will resize the image data to 50x50, transform it to tensors, and normalize it between -1 and 1. We apply the transformations to the VehicleDatset object, 'dataset', and pass it into the torch DataLoader class with batch size 100 and shuffle set to True.

In [ ]:
#import torch
from torch import nn, optim
from torch.autograd.variable import Variable
#from torchvision import transforms
#from PIL import Image

#import import_ipynb
#from CustomDataset import VehicleDataset

from utils import Logger # this is used to visualize generator data and print status logs during training

trsfm = transforms.Compose([
        transforms.Resize((50,50),interpolation=Image.NEAREST),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 


dataset = VehicleDataset(main_dir=newDir, transform=trsfm)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# check if cuda is available and use it in the '.to(device)' methods used later if it is. If not, use cpu in the method.
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

#number of batches
num_batches = len(data_loader)
print('number of batches:', num_batches)

from itertools import islice

# verifying contents of data_loader is as expected
print('Using DataLoader to show data: ')
for i, img in enumerate(islice(data_loader, 5)):
    print('Batch:', i, ',Size:', img.size(), '\nTensor: ', img[0,:,0:3,0:3])

Defining the Discriminator neural network.

In [ ]:
class DiscriminatorNet(torch.nn.Module):
    """
    A three hidden-layer discriminative neural network
    """
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        n_features = 7500
        n_out = 1
        
        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(256, n_out),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
    
discriminator = DiscriminatorNet()
discriminator.to(device)

print(discriminator)

Methods for transforming 4d image tensors to flattened image tensors and vice versa.

In [ ]:
def images_to_tensors(images):
    return images.view(images.size(0), 7500)

def tensors_to_images(tensors):
    return tensors.view(tensors.size(0), 3, 50, 50)

Defining the Generator neural network.

In [ ]:
class GeneratorNet(torch.nn.Module):
    """
    A three hidden-layer generative neural network
    """
    def __init__(self):
        super(GeneratorNet, self).__init__()
        n_features = 1000
        n_out = 7500
        
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.LeakyReLU(0.2)
        )
        self.hidden1 = nn.Sequential(            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )
        
        self.out = nn.Sequential(
            nn.Linear(1024, n_out),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
    
generator = GeneratorNet()
generator.to(device)

# Loads pretrained generator weights
# #path = os.path.join(os.getcwd(), 'data', 'models')

# if os.path.exists(path):
#     GeneratorNet.load_state_dict(torch.load(path))

print(generator)

Method for creating random noise corresponding to shape (batch size, number of inputs to generator).

In [ ]:
def noise(size):
    '''
    Generates a 1-d vector of gaussian sampled random values
    '''
    n = Variable(torch.randn(size, 1000))
    return n.to(device)

Initializing adam optimizers for discriminator and generator networks with learning rate of 0.0002 and Binary Cross Entropy loss function.

In [ ]:
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

loss = nn.BCELoss()

Methods for creating targets of ones for real data and targets of zeroes for fake generator output.

In [ ]:
def ones_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    return data.to(device)

def zeros_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    return data.to(device)


Function for training Discriminator network.

In [ ]:
def train_discriminator(optimizer, real_data, fake_data):
    N = real_data.size(0)
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = discriminator(real_data)
    # Calculate error and backpropagate
    error_real = loss(prediction_real, ones_target(N) )
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, zeros_target(N))
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error and predictions for real and fake inputs
    return error_real + error_fake, prediction_real, prediction_fake

Function for training Generator network.

In [ ]:
def train_generator(optimizer, fake_data):
    N = fake_data.size(0)
    # Reset gradients
    optimizer.zero_grad()
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    # Calculate error and backpropagate
    error = loss(prediction, ones_target(N))
    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

Creating 16 samples of noise data to visualize 16 generated images using tensorboard logger.

In [ ]:
num_test_samples = 16
test_noise = noise(num_test_samples)

Finally, training the GAN and displaying progress using Logger functions.

In [ ]:
# Create logger instance
logger = Logger(model_name='VGAN', data_name='Parked Vehicles')

# Total number of epochs to train
num_epochs = 200

g_losses = []
d_losses = []

for epoch in range(num_epochs):
    
    for n_batch, (real_batch) in enumerate(data_loader):
        N = real_batch.size(0)
        # 1. Train Discriminator
        real_data = Variable(images_to_tensors(real_batch)).to(device)
        # Generate fake data and detach 
        # (so gradients are not calculated for generator)
        fake_data = generator(noise(N)).detach()
        # Train D
        d_error, d_pred_real, d_pred_fake = \
              train_discriminator(d_optimizer, real_data, fake_data)

        # 2. Train Generator
        # Generate fake data
        fake_data = generator(noise(N))
        # Train G
        g_error = train_generator(g_optimizer, fake_data)
        # Log batch error
        logger.log(d_error, g_error, epoch, n_batch, num_batches)
        
        # Display Progress every few batches
        if (n_batch) % 50 == 0: 
            test_images = tensors_to_images(generator(test_noise)).cpu().detach()
            test_images = test_images.data
            logger.log_images(
                test_images, num_test_samples, 
                epoch, n_batch, num_batches
            );
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, n_batch, num_batches,
                d_error, g_error, d_pred_real, d_pred_fake
            )
            # record generator and discriminator losses every 50 batch for plotting 
            g_losses.append(g_error/n_batch)
            d_losses.append(d_error/n_batch)
        
        # saves generator weights every 50 epochs
#         if(epoch > 49):
#             path = os.path.join(os.getcwd(), 'data', 'models')
#             logger._make_dir(path)
#             if (epoch) % 50 == 0:
#                 torch.save(GeneratorNet.state_dict(),
#                            '%s\G_epoch_%s' % (path,epoch))
        
print('\nTraining Finished')

Plotting Generator and Discriminator losses per epoch.

In [ ]:
from matplotlib import pyplot as plt

plt.plot(g_losses, label='Generator_Losses')
plt.plot(d_losses, label='Discriminator Losses')
plt.legend()
plt.savefig('loss.png')